# Add Articles to DB
    1. Get the list of new articles from CSV file
    2. Get article data from crossref
    3. Add articles to DB

In [3]:
# Libraries
# library containign functions that read and write to csv files
import lib.handle_csv as csvh
# library for getting data from crossref
import lib.crossref_api as cr_api
# library for mapping json data
import lib.handle_json as hnd_json



input_file = "processed_csv/AddNewArticles202002.csv"

csv_articles, _ = csvh.get_csv_data(input_file,'Num')


cr_articles = {} # Num (from CHUK), DOI, type, and other fiedls from CR.
article_columns=["Num"]
# list of article authors from cross ref
cr_authors = {} # AuthorNum, Firstname, Middle name, Last Name
author_columns = ["AuthorNum", "FirstName", "MiddleName", "LastName"]
# list of article-author links
cr_article_authour_link = {} # AuthorNum, DOI
article_authour_columns = ["DOI","AuthorNum"]

for art_num in csv_articles:
    article_title = csv_articles[art_num]['Title']
    doi_text = csv_articles[art_num]['DOI']
    # get article data from CrossRef
    while True:
        try_n = 1
        article_data = cr_api.getBibData(doi_text)
        if article_data != {}:
            break
        else:  
            print("****************************************************")
            print("retry", try_n, article_data)
            try_n += 1
    # names for article data columns include/ignore
    data_keys = list(article_data.keys())
    for key in data_keys:
        if not (key in article_columns) and \
           key not in ['author', 'assertion', 'indexed', 'funder',
                       'content-domain','created','update-policy', 'source',
                       'is-referenced-by-count','prefix','member',
                       'reference','original-title','language','deposited',
                       'score', 'subtitle', 'short-title', 'issued',
                       'alternative-id','relation','ISSN','container-title-short']:
            article_columns.append(key)
    new_row = {}
    # map author data
    for key in article_columns:
        if key == 'Num':
            new_row['NumUKCH'] = art_num
        else:
            if key in article_data.keys():
                new_row[key] = hnd_json.jsonToPlainText(article_data[key])
    print(new_row)
    # map 

{'NumUKCH': 3030, 'reference-count': 46, 'publisher': 'Royal Society of Chemistry (RSC)', 'issue': '2', 'license': 'http://creativecommons.org/licenses/by/3.0/', 'abstract': '<p>Protagonists in the standard SCR reaction have been caught in the act by modulation excitation IR &amp; XAFS spectroscopy.</p>', 'DOI': '10.1039/c9sc04905c', 'type': 'article-journal', 'page': '447-455', 'title': 'Detection of key transient Cu intermediates in SSZ-13 during NH3-SCR deNOx by modulation excitation IR spectroscopy', 'volume': '11', 'published-online': [2020], 'container-title': 'Chemical Science', 'link': 'http://pubs.rsc.org/en/content/articlepdf/2020/SC/C9SC04905C', 'references-count': 46, 'journal-issue': '2', 'URL': 'http://dx.doi.org/10.1039/c9sc04905c', 'subject': None}
{'NumUKCH': 3035, 'reference-count': 80, 'publisher': 'Wiley', 'issue': '5', 'license': 'http://creativecommons.org/licenses/by/4.0/', 'DOI': '10.1002/cctc.201901955', 'type': 'article-journal', 'page': '1491-1503', 'title': 